In [ ]:
pip install anthropic

In [ ]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

In [ ]:
import anthropic

client = anthropic.Anthropic(
    api_key=CLAUDE_API_KEY,
)

message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)

In [ ]:
response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature, either 'celsius' or 'fahrenheit'"
                    }
                },
                "required": ["location"]
            }
        }
    ],
    messages=[
        {
            "role": "user",
            "content": "What's the weather like in San Francisco?"
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "<thinking>I need to use get_weather, and the user wants SF, which is likely San Francisco, CA.</thinking>"
                },
                {
                    "type": "tool_use",
                    "id": "toolu_01A09q90qw90lq917835lq9",
                    "name": "get_weather",
                    "input": {"location": "San Francisco, CA", "unit": "celsius"}
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": "toolu_01A09q90qw90lq917835lq9", # from the API response
                    "content": "65 degrees" # from running your tool
                }
            ]
        }
    ]
)

print(response)

Message(id='msg_01Hm2E7aDUw1mmHbGEiUW7PX', content=[TextBlock(citations=None, text='<thinking>\nI notice there\'s a discrepancy in the result. The function returned "65 degrees" without specifying the unit. According to the function parameters, I requested celsius as the unit, but 65°C (149°F) would be extremely hot and unrealistic for San Francisco weather. \n\nMore likely, the API returned the temperature in Fahrenheit despite my request for celsius, which is a common default for US locations. 65°F (about 18°C) is a much more reasonable temperature for San Francisco.\n\nI should clarify this to the user, but present the information as given by the API.\n</thinking>\n\nThe weather in San Francisco, CA is currently 65 degrees. The API didn\'t specify the unit in its response, but given that this is a US location and the temperature value, this is most likely 65°F (approximately 18°C), which is typical mild weather for San Francisco.', type='text')], model='claude-opus-4-20250514', role

In [ ]:
response.content[0].text

'<thinking>\nI notice there\'s a discrepancy in the result. The function returned "65 degrees" without specifying the unit. According to the function parameters, I requested celsius as the unit, but 65°C (149°F) would be extremely hot and unrealistic for San Francisco weather. \n\nMore likely, the API returned the temperature in Fahrenheit despite my request for celsius, which is a common default for US locations. 65°F (about 18°C) is a much more reasonable temperature for San Francisco.\n\nI should clarify this to the user, but present the information as given by the API.\n</thinking>\n\nThe weather in San Francisco, CA is currently 65 degrees. The API didn\'t specify the unit in its response, but given that this is a US location and the temperature value, this is most likely 65°F (approximately 18°C), which is typical mild weather for San Francisco.'

In [ ]:
initial_response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=[
        {
        "name": "get_stock_price",
        "description": "Retrieves the current stock price for a given ticker symbol. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or company.",
        "input_schema": {
            "type": "object",
            "properties": {
            "ticker": {
                "type": "string",
                "description": "The stock ticker symbol, e.g. AAPL for Apple Inc."
            }
            },
            "required": ["ticker"]
        }
        },
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature, either 'celsius' or 'fahrenheit'"
                    }
                },
                "required": ["location"]
            }
        },
        {
            "name": "get_time",
            "description": "Get the current time in a given time zone",
            "input_schema": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The IANA time zone name, e.g. America/Los_Angeles"
                    }
                },
                "required": ["timezone"]
            }
        }
    ],
    messages=[
        {
            "role": "user",
            "content": "What is the latest Tesla stock price?"
        }
    ]
)
print(initial_response)

Message(id='msg_017UnGGMSchKR9A2gQYfFoYQ', content=[TextBlock(citations=None, text="I'll get the latest Tesla stock price for you.", type='text'), ToolUseBlock(id='toolu_01QVNrC1h2sVCqzn5FYbbiux', input={'ticker': 'TSLA'}, name='get_stock_price', type='tool_use')], model='claude-opus-4-20250514', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=674, output_tokens=69, server_tool_use=None, service_tier='standard'))


In [ ]:
initial_response.stop_reason

'tool_use'